#### Use this for sentiment analysis with Logistic Regression and Naive Bayes
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_Sentiment_Analysis.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/1_IMPORTANT_Capstone-Project-master/Capstone-Project-master/Capstone_Project_KNN_RecommenderSystem.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Naive%20Bayes/Amazon%20reviews_Naive%20Bayes.ipynb
- http://localhost:8888/notebooks/_PythonNotebooks/_Course8_Deployment_Capstone/2_Amazon-Product-Recommender-System-master/Recommender%20System/Recommender%20System.ipynb

In [23]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

In [24]:
import re

In [38]:
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn import neighbors
import matplotlib.pyplot as plt


In [51]:
df = pd.read_csv("sample30.csv")

In [52]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [53]:
print(df.columns)
print(df.shape)

Index(['id', 'brand', 'categories', 'manufacturer', 'name', 'reviews_date',
       'reviews_didPurchase', 'reviews_doRecommend', 'reviews_rating',
       'reviews_text', 'reviews_title', 'reviews_userCity',
       'reviews_userProvince', 'reviews_username', 'user_sentiment'],
      dtype='object')
(30000, 15)


In [54]:
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


In [55]:
#Tokenize the review text to words
reg = re.compile('[^a-z]+')
def tokenizeReview(text):
    text = text.lower()
    text = reg.sub(' ', text).strip()
    return text

In [56]:
df['reviews_tokenized'] = df['reviews_text'].apply(tokenizeReview)

In [57]:
df.shape

(30000, 16)

In [58]:
df = df.drop(['reviews_text'], axis=1)

In [59]:
df.shape

(30000, 15)

In [60]:
df['reviews_tokenized'].head(5)

0    i love this album it s very good more to the h...
1    good flavor this review was collected as part ...
2                                          good flavor
3    i read through the reviews on here before look...
4    my husband bought this gel for us the gel caus...
Name: reviews_tokenized, dtype: object

In [65]:
df['user_sentiment'].value_counts()

Positive    26632
Negative     3367
Name: user_sentiment, dtype: int64

In [68]:
def senti(txt):
    if str(txt).lower() == 'positive':
        return 1
    else:
        return 0
    
X = df['reviews_tokenized']
y = df['user_sentiment'].apply(senti)

In [69]:
y

0        1
1        1
2        1
3        0
4        0
        ..
29995    1
29996    1
29997    1
29998    1
29999    1
Name: user_sentiment, Length: 30000, dtype: int64

In [70]:
tfidf = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', stop_words= 'english')

In [71]:
def model_fit(X, y, feature_model,ml_model,coef_show=1):
    
    X_features = feature_model.fit_transform(X)
    print('# features: {}'.format(X_features.shape[1]))
    X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size = 0.2, random_state=43)
    clf = ml_model.fit(X_train, y_train)
    clf_pred = clf.predict(X_test)
    accuracy = clf.score(X_test, y_test)
    model_performance = classification_report(y_test, clf_pred)
    print ('accuracy of the model: ', accuracy)
    print('')
    print(model_performance)
    
    if coef_show == 1: 
        w = feature_model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('Top 10 positive features (variables)')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('Top 10 negative features (variables)')        
        print(coeff_df.tail(20).to_string(index=False))
       

In [72]:
model_fit(X, y, tfidf,LogisticRegression(),coef_show=1)

# features: 1000
accuracy of the model:  0.9221666666666667

              precision    recall  f1-score   support

           0       0.81      0.38      0.52       660
           1       0.93      0.99      0.96      5340

    accuracy                           0.92      6000
   macro avg       0.87      0.69      0.74      6000
weighted avg       0.92      0.92      0.91      6000


Top 10 positive features (variables)
      Word  Coefficient
     great    13.299884
      love    11.054383
      best     8.156580
      good     7.800511
      easy     7.735733
     clean     7.719841
    better     5.808192
     loved     5.688351
      nice     5.268984
 excellent     5.121848
   awesome     4.898837
   perfect     4.861898
  favorite     4.555234
   enjoyed     4.373794
   amazing     4.364436
      free     3.939587
 wonderful     3.916948
    really     3.575024
     handy     3.529437
     happy     3.373818

Top 10 negative features (variables)
          Word  Coefficient
    